In [5]:
import math
import os
from pathlib import Path
from typing import Callable, Optional
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm
from wilds import get_dataset

from models import WaterbirdResNet18, SPDTwoLayerFC
from spd.run_spd import get_lr_schedule_fn, get_lr_with_warmup
from spd.hooks import HookedRootModule
from spd.log import logger
from spd.models.base import SPDModel
from spd.module_utils import (
    get_nested_module_attr,
    collect_nested_module_attrs,
)
from spd.types import Probability
from spd.utils import set_seed
from train_resnet import WaterbirdsSubset



In [6]:
from run_spd import load_config_from_yaml, run_spd_waterbird
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Suppose you want to rerun trial #7
config = load_config_from_yaml("optuna_trials/trial_27_config.yaml")
final_loss = run_spd_waterbird(config, device)
print("Final Loss from rerun:", final_loss)


2025-04-16 07:48:47 - INFO - Running SPD Waterbird with manual teacher caching, config=seed=27 batch_size=64 steps=966 lr=0.00017688154427024347 print_freq=50 save_freq=None out_dir=None distill_coeff=0.8673527736719934 param_match_coeff=2.6565345984830224 alpha_condition=1.0 cond_coeff=0.5058424630977391 C=40 m_fc1=16 m_fc2=16 lr_schedule='constant' lr_exponential_halflife=None lr_warmup_pct=0.0 unit_norm_matrices=False schatten_coeff=None schatten_pnorm=None teacher_ckpt='checkpoints/waterbird_resnet18_best.pth' topk=2.4825412692902824 batch_topk=True topk_recon_coeff=0.11381880979184406 distil_from_target=True lp_sparsity_coeff=0.009538420162649474 pnorm=2.0 attribution_type='gradient'
/workspace/apd/spd/experiments/waterbirds_resnet/run_spd.py:521: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during 

Total dataset size: 11788


2025-04-16 07:48:48 - INFO - Step 0 | total=505.3057 distill=12.7612, param=490.7272, topk=1.4665, lp=0.0000, cond=0.3507, lr=1.77e-04
2025-04-16 07:49:01 - INFO - Step 50 | total=323.1152 distill=0.0100, param=321.4576, topk=1.3039, lp=0.0076, cond=0.3360, lr=1.77e-04
2025-04-16 07:49:15 - INFO - Step 100 | total=139.9305 distill=0.0018, param=139.3091, topk=0.3530, lp=0.0191, cond=0.2474, lr=1.77e-04
2025-04-16 07:49:28 - INFO - Step 150 | total=40.2275 distill=0.0034, param=39.8569, topk=0.0847, lp=0.0161, cond=0.2663, lr=1.77e-04
2025-04-16 07:49:41 - INFO - Step 200 | total=10.0520 distill=0.0010, param=9.7290, topk=0.0678, lp=0.0131, cond=0.2411, lr=1.77e-04
2025-04-16 07:49:55 - INFO - Step 250 | total=2.9295 distill=0.0335, param=2.5366, topk=0.0741, lp=0.0116, cond=0.2737, lr=1.77e-04
2025-04-16 07:50:08 - INFO - Step 300 | total=1.1580 distill=0.0410, param=0.7583, topk=0.0458, lp=0.0124, cond=0.3006, lr=1.77e-04
2025-04-16 07:50:21 - INFO - Step 350 | total=0.6100 distill=0.

Final Loss from rerun: None


In [7]:
import torch
import torch.nn as nn

# Define a combined model that uses the pretrained ResNet backbone 
# but replaces the FC layers with your SPD module
class CombinedWaterbirdModel(nn.Module):
    def __init__(self, resnet_model, spd_model):
        super().__init__()
        # Use only the feature extractor part of the ResNet18 model
        self.features = resnet_model.features
        # Use the SPD model for the fully connected part
        self.spd_model = spd_model
        
    def forward(self, x):
        # Extract features using ResNet backbone
        feats = self.features(x)               # [B, 512, 1, 1]
        feats = feats.flatten(start_dim=1)     # [B, 512]
        
        # Pass features to SPD module
        out = self.spd_model(feats)
        return out

# 1. Load the pretrained ResNet model
resnet_model = WaterbirdResNet18(num_classes=2, hidden_dim=512)
resnet_ckpt_path = "checkpoints/waterbird_resnet18_best.pth"

# The file showed the checkpoint has a different structure - it's a dictionary
resnet_checkpoint = torch.load(resnet_ckpt_path, map_location="cpu")
resnet_model.load_state_dict(resnet_checkpoint['model_state_dict'])
resnet_model.eval()

# 2. Load the SPD model 
spd_model = SPDTwoLayerFC(
    in_features=512,
    hidden_dim=512,
    num_classes=2,
    C=40,
    m_fc1=16,
    m_fc2=16,
)

# Load SPD state dict
spd_ckpt_path = "optuna_trials/trial_29_spd.pth"
spd_state_dict = torch.load(spd_ckpt_path, map_location="cpu")
spd_model.load_state_dict(spd_state_dict)
spd_model.eval()

# 3. Create the combined model
combined_model = CombinedWaterbirdModel(resnet_model, spd_model)
combined_model.eval()

# To use for inference:
# with torch.no_grad():
#     output = combined_model(input_image)

/tmp/ipykernel_1840111/1031647141.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet_checkpoint = torch.load(resnet_ckpt_path, map_location="cpu")
/tmp/ipykernel_18

CombinedWaterbirdModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [8]:
spd_model.fc1.B.shape

torch.Size([40, 16, 512])

In [3]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import torchvision.transforms as T
from wilds import get_dataset
import torch.nn.functional as F

# First, setup validation dataset
waterbird_dataset = get_dataset(dataset="waterbirds", download=False)
dataset_size = len(waterbird_dataset)
print(f"Total dataset size: {dataset_size}")

# Get indices
all_indices = np.arange(dataset_size)
np.random.shuffle(all_indices)
train_indices = all_indices[:2000].tolist()
val_indices = all_indices[2000:3000].tolist()  # Taking 1000 samples after the 2000th index

# Setup validation transform
val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

# Create validation subset
val_subset = WaterbirdsSubset(
    waterbird_dataset, 
    indices=val_indices,
    transform=val_transform
)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

# Load the combined model
# 1. Load ResNet
resnet_model = WaterbirdResNet18(num_classes=2, hidden_dim=512)
resnet_ckpt_path = "checkpoints/waterbird_resnet18_best.pth"
resnet_checkpoint = torch.load(resnet_ckpt_path, map_location="cpu")
resnet_model.load_state_dict(resnet_checkpoint['model_state_dict'])
resnet_model.eval()

# 2. Load SPD model
spd_model = SPDTwoLayerFC(
    in_features=512,
    hidden_dim=512,
    num_classes=2,
    C=40,
    m_fc1=16,
    m_fc2=16,
)
spd_ckpt_path = "waterbird_spd_out/waterbird_spd_final.pth"
spd_state_dict = torch.load(spd_ckpt_path, map_location="cpu")
spd_model.load_state_dict(spd_state_dict)
spd_model.eval()

# 3. Create combined model
class CombinedWaterbirdModel(nn.Module):
    def __init__(self, resnet_model, spd_model):
        super().__init__()
        self.features = resnet_model.features
        self.spd_model = spd_model
        
    def forward(self, x):
        feats = self.features(x)
        feats = feats.flatten(start_dim=1)
        out = self.spd_model(feats)
        return out

# Create the combined model and move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
combined_model = CombinedWaterbirdModel(resnet_model, spd_model).to(device)
combined_model.eval()

# Evaluate on validation set
correct = 0
total = 0
metadata_correct = {}  # For analysis by metadata
metadata_total = {}

print(f"Evaluating combined model on {len(val_indices)} validation samples...")

with torch.no_grad():
    for inputs, labels, metadata in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = combined_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        # Update overall stats
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f"Combined model accuracy on validation set: {accuracy:.2f}%")

Total dataset size: 11788


/tmp/ipykernel_1246640/3955101530.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet_checkpoint = torch.load(resnet_ckpt_path, map_location="cpu")
/tmp/ipykernel_12

Evaluating combined model on 1000 validation samples...
Combined model accuracy on validation set: 71.00%


In [4]:
import torch
from wilds import get_dataset
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset
import numpy as np
from tqdm import tqdm

# Get the dataset
dataset = get_dataset(dataset="waterbirds", download=False)

# Create transform
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

# Define a function to get samples by group
def get_group_indices(dataset, bird_type, background):
    """
    Get indices of samples where:
    - bird_type: 0 for landbird, 1 for waterbird
    - background: 0 for land, 1 for water
    """
    indices = []
    for i in range(len(dataset)):
        x, y, metadata = dataset[i]
        # y is the bird type, metadata[1] is the background type
        if y == bird_type and metadata[0] == background:
            indices.append(i)
    return indices

# Get indices for each group
landbird_land = get_group_indices(dataset, 0, 0)  # Majority group
waterbird_water = get_group_indices(dataset, 1, 1)  # Majority group
landbird_water = get_group_indices(dataset, 0, 1)  # Minority group
waterbird_land = get_group_indices(dataset, 1, 0)  # Minority group

print(f"Landbirds on land: {len(landbird_land)} samples")
print(f"Waterbirds on water: {len(waterbird_water)} samples")
print(f"Landbirds on water: {len(landbird_water)} samples")  # This should be smaller
print(f"Waterbirds on land: {len(waterbird_land)} samples")  # This should be smaller

# Function to evaluate model on a specific group
def evaluate_group(model, dataset, indices, transform, device, batch_size=32):
    subset = Subset(dataset, indices)
    
    # Create a custom dataset that applies the transform
    class TransformSubset:
        def __init__(self, subset, transform):
            self.subset = subset
            self.transform = transform
            
        def __len__(self):
            return len(self.subset)
            
        def __getitem__(self, idx):
            x, y, metadata = self.subset[idx]
            if self.transform:
                x = self.transform(x)
            return x, y, metadata
    
    # Create the loader
    loader = DataLoader(
        TransformSubset(subset, transform),
        batch_size=batch_size,
        shuffle=False
    )
    
    # Evaluate
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels, metadata in tqdm(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total if total > 0 else 0
    return accuracy

# Now you can evaluate both your models on each group
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate original ResNet model
print("\nEvaluating original ResNet model:")
resnet_model = resnet_model.to(device)
landbird_land_acc = evaluate_group(resnet_model, dataset, landbird_land, transform, device)
waterbird_water_acc = evaluate_group(resnet_model, dataset, waterbird_water, transform, device)
landbird_water_acc = evaluate_group(resnet_model, dataset, landbird_water, transform, device)
waterbird_land_acc = evaluate_group(resnet_model, dataset, waterbird_land, transform, device)

print(f"Landbirds on land: {landbird_land_acc:.2f}%")
print(f"Waterbirds on water: {waterbird_water_acc:.2f}%")
print(f"Landbirds on water: {landbird_water_acc:.2f}%")
print(f"Waterbirds on land: {waterbird_land_acc:.2f}%")

# Evaluate combined model
print("\nEvaluating combined model:")
combined_model = combined_model.to(device)
landbird_land_acc = evaluate_group(combined_model, dataset, landbird_land, transform, device)
waterbird_water_acc = evaluate_group(combined_model, dataset, waterbird_water, transform, device)
landbird_water_acc = evaluate_group(combined_model, dataset, landbird_water, transform, device)
waterbird_land_acc = evaluate_group(combined_model, dataset, waterbird_land, transform, device)

print(f"Landbirds on land: {landbird_land_acc:.2f}%")
print(f"Waterbirds on water: {waterbird_water_acc:.2f}%")
print(f"Landbirds on water: {landbird_water_acc:.2f}%")
print(f"Waterbirds on land: {waterbird_land_acc:.2f}%")

Landbirds on land: 6220 samples
Waterbirds on water: 1832 samples
Landbirds on water: 2905 samples
Waterbirds on land: 831 samples

Evaluating original ResNet model:


100%|██████████| 26/26 [00:03<00:00,  7.02it/s]


Landbirds on land: 98.26%
Waterbirds on water: 86.08%
Landbirds on water: 25.44%
Waterbirds on land: 4.93%

Evaluating combined model:


100%|██████████| 26/26 [00:03<00:00,  7.14it/s]

Landbirds on land: 98.31%
Waterbirds on water: 85.81%
Landbirds on water: 25.99%
Waterbirds on land: 4.81%


In [5]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import torchvision.transforms as T
from wilds import get_dataset
import torch.nn.functional as F

# --- Same data-loading code as before ---
# (omitted for brevity)

# --- Same checkpoint loading for resnet_model and spd_model ---
# (omitted for brevity)

# Evaluate on validation set, but ablate circuit #0
correct = 0
total = 0

print(f"Evaluating combined model on {len(val_indices)} validation samples, "
      "ablating circuit #0...")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.to(device).eval()
spd_model.to(device).eval()

with torch.no_grad():
    for inputs, labels, metadata in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 1) Extract features from ResNet trunk
        feats = resnet_model.features(inputs)
        feats = feats.flatten(start_dim=1)  # shape [batch_size, 512]

        # 2) Build the topk_mask that ablates circuit #0
        batch_size = feats.size(0)
        topk_mask = torch.ones((batch_size, spd_model.C), dtype=torch.bool, device=device)
        topk_mask[:, 0] = False  # Turn off circuit #0 for every example

        # 3) Forward pass through SPD with ablation
        outputs = spd_model(feats, topk_mask=topk_mask)

        # 4) Compute predictions
        _, predicted = torch.max(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Ablating circuit #0 => Accuracy on validation set: {accuracy:.2f}%")


Evaluating combined model on 1000 validation samples, ablating circuit #0...
Ablating circuit #0 => Accuracy on validation set: 71.10%


In [9]:
import torch
from wilds import get_dataset
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset
import numpy as np
from tqdm import tqdm

# Get the dataset
dataset = get_dataset(dataset="waterbirds", download=False)

# Create transform
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

# Define a function to get samples by group
def get_group_indices(dataset, bird_type, background):
    """
    Get indices of samples where:
    - bird_type: 0 for landbird, 1 for waterbird
    - background: 0 for land, 1 for water
    """
    indices = []
    for i in range(len(dataset)):
        x, y, metadata = dataset[i]
        # y is the bird type, metadata[0] is the background type
        if y == bird_type and metadata[0] == background:
            indices.append(i)
    return indices

# Get indices for each group
landbird_land = get_group_indices(dataset, 0, 0)  # Majority group
waterbird_water = get_group_indices(dataset, 1, 1)  # Majority group
landbird_water = get_group_indices(dataset, 0, 1)  # Minority group
waterbird_land = get_group_indices(dataset, 1, 0)  # Minority group

print(f"Landbirds on land: {len(landbird_land)} samples")
print(f"Waterbirds on water: {len(waterbird_water)} samples")
print(f"Landbirds on water: {len(landbird_water)} samples")
print(f"Waterbirds on land: {len(waterbird_land)} samples")

# Modified function to evaluate model on a specific group with optional circuit ablation
def evaluate_group(resnet_model, spd_model, dataset, indices, transform, device, 
                  batch_size=32, ablate_circuits=None):
    """
    Evaluate model on a specific group with optional circuit ablation
    
    Args:
        resnet_model: Feature extractor model
        spd_model: SPD model with circuits
        dataset: Dataset object
        indices: Indices to evaluate on
        transform: Image transform
        device: Device to run on
        batch_size: Batch size for evaluation
        ablate_circuits: List of circuit indices to ablate (set to None for no ablation)
    
    Returns:
        accuracy: Accuracy on the evaluated group
    """
    subset = Subset(dataset, indices)
    
    # Create a custom dataset that applies the transform
    class TransformSubset:
        def __init__(self, subset, transform):
            self.subset = subset
            self.transform = transform
            
        def __len__(self):
            return len(self.subset)
            
        def __getitem__(self, idx):
            x, y, metadata = self.subset[idx]
            if self.transform:
                x = self.transform(x)
            return x, y, metadata
    
    # Create the loader
    loader = DataLoader(
        TransformSubset(subset, transform),
        batch_size=batch_size,
        shuffle=False
    )
    
    # Evaluate
    correct = 0
    total = 0
    resnet_model.eval()
    spd_model.eval()
    
    with torch.no_grad():
        for inputs, labels, metadata in tqdm(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Extract features using ResNet
            feats = resnet_model.features(inputs)
            feats = feats.flatten(start_dim=1)  # [B, 512]
            
            # Create ablation mask if needed
            if ablate_circuits is not None and len(ablate_circuits) > 0:
                batch_size = feats.size(0)
                topk_mask = torch.ones((batch_size, spd_model.C), dtype=torch.bool, device=device)
                for circuit_idx in ablate_circuits:
                    topk_mask[:, circuit_idx] = False  # Turn off specified circuits
                
                # Forward pass through SPD with ablation
                outputs = spd_model(feats, topk_mask=topk_mask)
            else:
                # Normal forward pass through SPD without ablation
                outputs = spd_model(feats)
            
            # Compute predictions
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total if total > 0 else 0
    return accuracy

# Now you can evaluate both models on each group
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models (assuming you've already loaded them before)
# resnet_model = ... (your ResNet model)
# spd_model = ... (your SPD model)

# First, evaluate the standard model performance (without ablation)
print("\nEvaluating combined model (no ablation):")
landbird_land_acc = evaluate_group(resnet_model, spd_model, dataset, landbird_land, transform, device)
waterbird_water_acc = evaluate_group(resnet_model, spd_model, dataset, waterbird_water, transform, device)
landbird_water_acc = evaluate_group(resnet_model, spd_model, dataset, landbird_water, transform, device)
waterbird_land_acc = evaluate_group(resnet_model, spd_model, dataset, waterbird_land, transform, device)

print(f"Landbirds on land: {landbird_land_acc:.2f}%")
print(f"Waterbirds on water: {waterbird_water_acc:.2f}%")
print(f"Landbirds on water: {landbird_water_acc:.2f}%")
print(f"Waterbirds on land: {waterbird_land_acc:.2f}%")

# Now evaluate with circuit #0 ablated
ablate_circuits = [0]  # Ablate circuit #0
print(f"\nEvaluating combined model with circuit(s) {ablate_circuits} ablated:")
landbird_land_acc_abl = evaluate_group(resnet_model, spd_model, dataset, landbird_land, transform, device, ablate_circuits=ablate_circuits)
waterbird_water_acc_abl = evaluate_group(resnet_model, spd_model, dataset, waterbird_water, transform, device, ablate_circuits=ablate_circuits)
landbird_water_acc_abl = evaluate_group(resnet_model, spd_model, dataset, landbird_water, transform, device, ablate_circuits=ablate_circuits)
waterbird_land_acc_abl = evaluate_group(resnet_model, spd_model, dataset, waterbird_land, transform, device, ablate_circuits=ablate_circuits)

print(f"Landbirds on land: {landbird_land_acc_abl:.2f}%")
print(f"Waterbirds on water: {waterbird_water_acc_abl:.2f}%")
print(f"Landbirds on water: {landbird_water_acc_abl:.2f}%")
print(f"Waterbirds on land: {waterbird_land_acc_abl:.2f}%")

# Print the differences
print("\nAccuracy differences (ablated - normal):")
print(f"Landbirds on land: {landbird_land_acc_abl - landbird_land_acc:.2f}%")
print(f"Waterbirds on water: {waterbird_water_acc_abl - waterbird_water_acc:.2f}%")
print(f"Landbirds on water: {landbird_water_acc_abl - landbird_water_acc:.2f}%")
print(f"Waterbirds on land: {waterbird_land_acc_abl - waterbird_land_acc:.2f}%")

# You can easily test ablating multiple circuits
ablate_circuits = [0, 1, 2]  # Ablate circuits 0, 1, and 2
print(f"\nEvaluating combined model with circuit(s) {ablate_circuits} ablated:")
landbird_land_acc_multi = evaluate_group(resnet_model, spd_model, dataset, landbird_land, transform, device, ablate_circuits=ablate_circuits)
waterbird_water_acc_multi = evaluate_group(resnet_model, spd_model, dataset, waterbird_water, transform, device, ablate_circuits=ablate_circuits)
landbird_water_acc_multi = evaluate_group(resnet_model, spd_model, dataset, landbird_water, transform, device, ablate_circuits=ablate_circuits)
waterbird_land_acc_multi = evaluate_group(resnet_model, spd_model, dataset, waterbird_land, transform, device, ablate_circuits=ablate_circuits)

print(f"Landbirds on land: {landbird_land_acc_multi:.2f}%")
print(f"Waterbirds on water: {waterbird_water_acc_multi:.2f}%")
print(f"Landbirds on water: {landbird_water_acc_multi:.2f}%")
print(f"Waterbirds on land: {waterbird_land_acc_multi:.2f}%")

Landbirds on land: 6220 samples
Waterbirds on water: 1832 samples
Landbirds on water: 2905 samples
Waterbirds on land: 831 samples

Evaluating combined model (no ablation):



 0%|          | 0/195 [00:00<?, ?it/s]

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

## Tests to confirm that this is working
### 1) Plotting distribution of attribution scores on samples where water is present

In [4]:
teacher_model

NameError: name 'teacher_model' is not defined

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as T
import numpy as np

# Imports from your own code
# (Adjust paths/modules as necessary—e.g., from spd.run_spd import calculate_attributions, etc.)
from models import WaterbirdResNet18
from spd.run_spd import calculate_attributions, SPDTwoLayerFC
from spd.utils import set_seed
from train_resnet import WaterbirdsSubset  # or your local subset wrapper
from wilds import get_dataset

def gather_gradient_attributions_for_water(
    spd_ckpt_path: str = "waterbird_spd_final.pth",
    teacher_ckpt_path: str = "checkpoints/waterbird_resnet18_best.pth",
    batch_size: int = 32,
    seed: int = 0,
    device: str = "cuda"
):
    """
    Loads teacher (ResNet) and SPD final-layers, filters Waterbirds to only water background,
    then replicates the EXACT gradient-based attribution steps from run_spd_waterbird.py.
    Returns a list of [B, C] tensors, one per batch, where each entry is the subcomponent's
    gradient-based attribution score for that batch.
    """
    set_seed(seed)
    device = torch.device(device if torch.cuda.is_available() else "cpu")

    # 1) Load teacher (ResNet18) – must match how you trained SPD
    teacher_model = WaterbirdResNet18(num_classes=2, hidden_dim=512)
    teacher_ckpt = torch.load(teacher_ckpt_path, map_location="cpu")
    if "model_state_dict" in teacher_ckpt:
        teacher_model.load_state_dict(teacher_ckpt["model_state_dict"], strict=False)
    else:
        teacher_model.load_state_dict(teacher_ckpt, strict=False)
    teacher_model.to(device)
    teacher_model.eval()

    # Separate the trunk and final layers from teacher
    teacher_trunk = teacher_model.features
    teacher_fc1 = teacher_model.fc1
    teacher_fc2 = teacher_model.fc2

    # 2) Load SPD final-layers (same arch as your config)
    spd_fc = SPDTwoLayerFC(
        in_features=512,
        hidden_dim=512,
        num_classes=2,
        C=40,      # must match your SPD config
        m_fc1=16,  # likewise
        m_fc2=16
    )
    spd_fc.load_state_dict(torch.load(spd_ckpt_path, map_location="cpu"))
    spd_fc.to(device)
    spd_fc.eval()

    # 3) Filter Waterbirds for background == "water".
    #    Depending on your WILDS version, the metadata indexing might differ:
    #       - Some store water/land at meta[2], others at meta[1].
    #       - Double-check in your code and adjust if needed.
    waterbird_ds = get_dataset("waterbirds", download=False)
    water_indices = []
    for idx in range(len(waterbird_ds)):
        _, _, meta = waterbird_ds[idx]
        # For standard WILDS waterbirds, meta = [y, place, background].
        # background==1 => water. If your code differs, adjust here.
        if meta[2] == 1:  
            water_indices.append(idx)

    transform = T.Compose([T.Resize((224,224)), T.ToTensor()])

    # Use your custom subset + transform
    water_subset = WaterbirdsSubset(waterbird_ds, water_indices, transform=transform)
    water_loader = DataLoader(water_subset, batch_size=batch_size, shuffle=False)

    # 4) Iterate over water images and replicate the run_spd logic for gradient attributions
    all_attributions = []  # will store one [B, C] attribution tensor per batch

    for (imgs, _, _) in water_loader:
        imgs = imgs.to(device)

        # 4a) Get trunk features
        # Exactly like run_spd, we feed plain images -> trunk -> flatten
        with torch.no_grad():
            feats = teacher_trunk(imgs)
            feats = feats.flatten(1)  # [B, 512]

        # 4b) Teacher forward pass with feats_with_grad so we can do gradient-based attributions
        feats_with_grad = feats.detach().clone().requires_grad_(True)

        teacher_cache = {}
        teacher_cache["fc1.hook_pre"] = feats_with_grad
        teacher_h_pre = teacher_fc1(feats_with_grad)
        teacher_cache["fc1.hook_post"] = teacher_h_pre

        teacher_h = torch.relu(teacher_h_pre)
        teacher_cache["fc2.hook_pre"] = teacher_h

        teacher_out = teacher_fc2(teacher_h)
        teacher_cache["fc2.hook_post"] = teacher_out

        # 4c) SPD forward pass with hooking
        spd_fc.reset_hooks()
        cache_dict, fwd_hooks, _ = spd_fc.get_caching_hooks()

        # no_grad is okay for SPD forward, since we only need the hooking of forward activations
        with torch.no_grad(), spd_fc.hooks(fwd_hooks, [], reset_hooks_end=True):
            spd_h_pre = spd_fc.fc1(feats)
            spd_h = torch.relu(spd_h_pre)
            spd_out = spd_fc.fc2(spd_h)

        # 4d) Gather SPD pre/post/comp
        pre_weight_acts = {}
        post_weight_acts = {}
        comp_acts = {}
        for k,v in cache_dict.items():
            if k.endswith("hook_pre"):
                pre_weight_acts[k] = v
            elif k.endswith("hook_post"):
                post_weight_acts[k] = v
            elif k.endswith("hook_component_acts"):
                # e.g. "fc1.hook_component_acts" => "fc1"
                layer_name = k.removesuffix(".hook_component_acts")
                comp_acts[layer_name] = v

        teacher_pre_acts = {k:v for k,v in teacher_cache.items() if k.endswith("hook_pre")}
        teacher_post_acts= {k:v for k,v in teacher_cache.items() if k.endswith("hook_post")}

        # 4e) Calculate gradient-based attributions wrt TEACHER OUT
        # Exactly as your run_spd does (the function does autograd.grad(teacher_out.sum(), ...)).
        # Ensure `attribution_type='gradient'`.
        # The teacher_out here is shape [B, 2], so .sum() is a scalar.
        attributions = calculate_attributions(
            model=spd_fc,
            input_x=feats,            # SPD used feats without grad
            out=spd_out,             # SPD's final output
            teacher_out=teacher_out, # We'll do grad wrt teacher_out
            pre_acts=teacher_pre_acts,
            post_acts=teacher_post_acts,
            component_acts=comp_acts,
            attribution_type="gradient"
        )
        # attributions => shape [B, C]

        # 4f) Store for later
        all_attributions.append(attributions.cpu())

    # 5) all_attributions is now a list of Tensors, one per batch
    #    You can cat them or keep them separate. For example, to combine them:
    all_attributions = torch.cat(all_attributions, dim=0)  # shape [N, C], all water images
    return all_attributions


if __name__ == "__main__":
    water_attr = gather_gradient_attributions_for_water(
        spd_ckpt_path="waterbird_spd_final.pth",
        teacher_ckpt_path="checkpoints/waterbird_resnet18_best.pth",
        batch_size=32,
        seed=0,
        device="cuda",
    )
    print("Gradient-based SPD attributions for water backgrounds:", water_attr.shape)
    # For example, you might save them:
    torch.save(water_attr, "water_attr.pt")


ImportError: cannot import name 'SPDTwoLayerFC' from 'spd.run_spd' (/workspace/apd/spd/run_spd.py)

In [3]:
img, _, meta = next(iter(loader))

NameError: name 'loader' is not defined